In [9]:
# --- Các thư viện bạn đã có ---
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns 
import matplotlib.pyplot as plt

# --- THÊM TOÀN BỘ CÁC DÒNG NÀY VÀO CELL 1 ---
from sklearn.ensemble import (
    RandomForestClassifier, 
    ExtraTreesClassifier, 
    AdaBoostClassifier, 
    GradientBoostingClassifier
)
from sklearn.linear_model import (
    LogisticRegression, 
    SGDClassifier, 
    RidgeClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

print("Tất cả thư viện (bao gồm nhiều model) đã được nạp!")

Tất cả thư viện (bao gồm nhiều model) đã được nạp!


In [2]:
# File paths
TRAIN_FILE = '../1_data/processed/train.csv'
VALID_FILE = '../1_data/processed/valid.csv'
TEST_FILE = '../1_data/processed/test.csv'

# Read data
df_train = pd.read_csv(TRAIN_FILE)
df_valid = pd.read_csv(VALID_FILE)
df_test = pd.read_csv(TEST_FILE)
labels_to_remove = ['DD'] 

df_train = df_train[~df_train['label'].isin(labels_to_remove)]
df_valid = df_valid[~df_valid['label'].isin(labels_to_remove)]
df_test = df_test[~df_test['label'].isin(labels_to_remove)]

print(f"Load {len(df_train)} row Train.")
print(f"Load {len(df_valid)} row Valid.")
print(f"Load {len(df_test)} row Test.")

# Show first 5 rows of training data
df_train.head()

Load 4654 row Train.
Load 1582 row Valid.
Load 743 row Test.


,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,A,0.0,0.0,0.0,0.074581,-0.073236,-0.021343,0.119197,-0.180595,-0.027051,...,-0.020520,-0.060482,-0.280165,-0.045432,-0.037218,-0.223027,-0.034378,-0.029876,-0.169899,-0.017204
1,O,0.0,0.0,0.0,0.259611,-0.112703,0.243912,0.388057,-0.231143,0.292713,...,-0.256559,0.240462,-0.551434,-0.266212,0.396560,-0.530241,-0.202974,0.454101,-0.465045,-0.141404
2,M,0.0,0.0,0.0,0.254779,0.099448,0.097950,0.327970,0.232818,0.109392,...,-0.254443,-0.108459,0.453366,-0.254068,-0.067516,0.431758,-0.175408,-0.060232,0.382438,-0.110514
3,Q,0.0,0.0,0.0,0.120284,0.106904,-0.377682,0.132300,0.281805,-0.490897,...,0.080319,-0.285603,0.362020,0.025871,-0.189271,0.301898,0.007764,-0.188433,0.244455,0.006331
4,U,0.0,0.0,0.0,0.171348,-0.062104,-0.224855,0.153004,-0.190746,-0.281793,...,-0.077436,-0.380044,-0.421300,-0.224172,-0.343827,-0.368488,-0.235969,-0.316802,-0.302838,-0.206465


In [3]:
# Tách cột 'label' ra làm 'y' (đáp án)
y_train = df_train['label']
y_valid = df_valid['label']
y_test = df_test['label']

# Bỏ cột 'label' để lấy 'X' (dữ liệu đầu vào)
X_train = df_train.drop('label', axis=1)
X_valid = df_valid.drop('label', axis=1)
X_test = df_test.drop('label', axis=1)

print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước X_valid: {X_valid.shape}")
print(f"Kích thước X_test: {X_test.shape}")

Kích thước X_train: (4654, 63)
Kích thước X_valid: (1582, 63)
Kích thước X_test: (743, 63)


In [4]:
# Tạo một 'cỗ máy' chuẩn hóa
scaler = StandardScaler()

# B1: "Học" thang đo từ DỮ LIỆU TRAIN
scaler.fit(X_train)

# B2: "Áp dụng" thang đo đó cho cả 3 tập
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

print("Đã chuẩn hóa (scale) dữ liệu.")

Đã chuẩn hóa (scale) dữ liệu.


In [12]:
# --- BẠN CÓ THỂ DÙNG CELL NÀY ĐỂ THAY THẾ CHO CELL 5, 6, VÀ 7 ---

# 1. Định nghĩa các model bạn muốn thử
models_to_train = {
# 1. Nhóm "Ensemble" (Thường mạnh nhất)
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1),#Nhanh
    #"GradientBoosting": GradientBoostingClassifier(random_state=42), # Chậm hơn nhiều RF >1p , nhưng đáng thử *loại
    #"AdaBoost": AdaBoostClassifier(random_state=42),#Độ chính xác thấp

    # 2. Nhóm "Neural Net" & "Hàng xóm" (Cần Scale)
    "MLPClassifier": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42),
    "KNeighbors": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "SVM": SVC(random_state=42), # Sẽ chạy RẤT LÂU

    # 3. Nhóm "Tuyến tính" (Nhanh, Cần Scale)
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1),
    "SGDClassifier": SGDClassifier(random_state=42, n_jobs=-1),
    "RidgeClassifier": RidgeClassifier(random_state=42),
    "LDA": LinearDiscriminantAnalysis(),

    # 4. Nhóm "Cơ bản" (Nhanh nhất)
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "GaussianNB": GaussianNB(),
}

# 2. Tạo một nơi để lưu kết quả và model tốt nhất
test_results = {}
best_model = None
best_model_name = ""
best_test_accuracy = 0.0

print("Bắt đầu quá trình huấn luyện và đánh giá nhiều model...\n")

# 3. Vòng lặp huấn luyện và đánh giá từng model
for name, model in models_to_train.items():
    
    print(f"--- Đang huấn luyện: {name} ---")
    
    # --- Tương đương Cell 5 ---
    # "Dạy" cho model bằng dữ liệu train đã chuẩn hóa
    model.fit(X_train_scaled, y_train)
    print(f"Huấn luyện {name} hoàn tất.")
    
    # --- Tương đương Cell 6 ---
    # Đánh giá trên tập Valid
    valid_preds = model.predict(X_valid_scaled)
    valid_accuracy = accuracy_score(y_valid, valid_preds)
    print(f"Kết quả VALID của {name}: {valid_accuracy * 100:.2f}%")
    
    # --- Tương đương Cell 7 ---
    # Đánh giá trên tập Test
    test_preds = model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, test_preds)
    print(f"Kết quả TEST của {name}: {test_accuracy * 100:.2f}%")
    
    # Lưu kết quả test
    test_results[name] = test_accuracy
    
    # Cập nhật model tốt nhất dựa trên kết quả TEST
    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        best_model = model
        best_model_name = name
        print(f"*** {name} là model tốt nhất hiện tại! ***")
    
    print("---------------------------------------\n")

print("--- SO SÁNH KẾT QUẢ TEST CUỐI CÙNG ---")
for name, acc in test_results.items():
    print(f"{name}: {acc * 100:.2f}%")
    
print(f"\nModel tốt nhất là: {best_model_name} với độ chính xác {best_test_accuracy * 100:.2f}%")

Bắt đầu quá trình huấn luyện và đánh giá nhiều model...

--- Đang huấn luyện: RandomForest ---
Huấn luyện RandomForest hoàn tất.
Kết quả VALID của RandomForest: 99.62%
Kết quả TEST của RandomForest: 98.92%
*** RandomForest là model tốt nhất hiện tại! ***
---------------------------------------

--- Đang huấn luyện: ExtraTrees ---
Huấn luyện ExtraTrees hoàn tất.
Kết quả VALID của ExtraTrees: 99.43%
Kết quả TEST của ExtraTrees: 99.19%
*** ExtraTrees là model tốt nhất hiện tại! ***
---------------------------------------

--- Đang huấn luyện: AdaBoost ---


c:\Users\Lenovo\Documents\Hoctap\Sign_language_recognition\Model_1\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Huấn luyện AdaBoost hoàn tất.
Kết quả VALID của AdaBoost: 13.91%
Kết quả TEST của AdaBoost: 17.63%
---------------------------------------

--- Đang huấn luyện: MLPClassifier ---
Huấn luyện MLPClassifier hoàn tất.
Kết quả VALID của MLPClassifier: 99.18%
Kết quả TEST của MLPClassifier: 99.19%
---------------------------------------

--- Đang huấn luyện: KNeighbors ---
Huấn luyện KNeighbors hoàn tất.
Kết quả VALID của KNeighbors: 98.10%
Kết quả TEST của KNeighbors: 98.25%
---------------------------------------

--- Đang huấn luyện: SVM ---
Huấn luyện SVM hoàn tất.
Kết quả VALID của SVM: 93.30%
Kết quả TEST của SVM: 93.67%
---------------------------------------

--- Đang huấn luyện: LogisticRegression ---
Huấn luyện LogisticRegression hoàn tất.
Kết quả VALID của LogisticRegression: 98.36%
Kết quả TEST của LogisticRegression: 98.38%
---------------------------------------

--- Đang huấn luyện: SGDClassifier ---
Huấn luyện SGDClassifier hoàn tất.
Kết quả VALID của SGDClassifier: 94.25%
Kế

In [13]:
# --- BẠN CÓ THỂ DÙNG CELL NÀY ĐỂ THAY THẾ CHO CELL 8 ---

# Tạo thư mục nếu chưa có
import os
os.makedirs('../4_models/', exist_ok=True)

# Đặt tên file
# Lưu model TỐT NHẤT và scaler
MODEL_PATH = f'../4_models/{best_model_name}_best_model.pkl' # Tên file sẽ động, vd: "SVM_best_model.pkl"
SCALER_PATH = '../4_models/scaler.pkl' # Scaler vẫn giữ nguyên

# Lưu file
if best_model is not None:
    joblib.dump(best_model, MODEL_PATH)
    joblib.dump(scaler, SCALER_PATH)
    
    print(f"Đã lưu model tốt nhất ({best_model_name}) vào: {MODEL_PATH}")
    print(f"Đã lưu scaler vào: {SCALER_PATH}")
else:
    print("Không có model nào để lưu.")

Đã lưu model tốt nhất (ExtraTrees) vào: ../4_models/ExtraTrees_best_model.pkl
Đã lưu scaler vào: ../4_models/scaler.pkl
